https://python.langchain.com/v0.2/docs/integrations/retrievers/milvus_hybrid_search/

## Milvus Connect & Collection Load

In [1]:
from pymilvus import connections, utility, FieldSchema, CollectionSchema, DataType, Collection

QUERY_MODEL = 'solar-embedding-1-large-query'
PASSAGE_MODEL = 'solar-embedding-1-large-passage'

DENSE_FIELD = "dense_vector"
DENSE_DIMENSION = 4096 
DENSE_INDEX = {"index_type" : "FLAT", "metric_type" : "IP"}

SPARSE_FILED = "sparse_vector"
SPARSE_INDEX = {"index_type" : "SPARSE_INVERTED_INDEX", "metric_type" : "IP"}

# Collection 접속 및 생성
def milvus_connect(collection_name):
    # Milvus 설정
    MILVUS_HOST = '127.0.0.1'
    MILVUS_PORT = '19530'     

    # Milvus 연결
    connections.connect(host=MILVUS_HOST, port=MILVUS_PORT)
    if connections:
        print("Milvus connected")
    else:
        exit()

    # 컬렉션 존재 여부 확인 및 생성
    if utility.has_collection(collection_name):
        collection = Collection(collection_name)
        print(f"Collection '{collection_name}' loaded.")
    else:
        print(f"Collection '{collection_name}' does not exist.")
        print(f"Create '{collection_name}' Collection.")
        
        field_args = [
            FieldSchema(name='id', dtype=DataType.VARCHAR, is_primary=True, max_length=100),        # split된 block의 고유 ID
            FieldSchema(name='title', dtype=DataType.VARCHAR, max_length=2000),
            FieldSchema(name='date', dtype=DataType.INT64),
            FieldSchema(name='content', dtype=DataType.VARCHAR, max_length=5000),
            FieldSchema(name='NER', dtype=DataType.VARCHAR, max_length=3000),
            FieldSchema(name=DENSE_FIELD, dtype=DataType.FLOAT_VECTOR, dim=DENSE_DIMENSION),
            FieldSchema(name=SPARSE_FILED, dtype=DataType.SPARSE_FLOAT_VECTOR)
        ]
        
        # 스키마 정의
        schema = CollectionSchema(fields=field_args)
        
        # 컬렉션 생성
        collection = Collection(name=collection_name, schema=schema)
        
        # 인덱스 생성
        collection.create_index("dense_vector", DENSE_INDEX)
        collection.create_index("sparse_vector", SPARSE_INDEX)

        collection.flush()

        print(f"Collection '{collection_name}' created and loaded.")
    # 생성된 컬렉션 반환
    return collection

# Collection 제거용
def drop_collection(collection_name):
    MILVUS_HOST = '127.0.0.1'
    MILVUS_PORT = '19530' 
    connections.connect(host=MILVUS_HOST, port=MILVUS_PORT)
    if utility.has_collection(collection_name):
        utility.drop_collection(collection_name)
        print(f"Drop {collection_name} Complete.")
    else:
        print(f"Can't find {collection_name}.")

In [3]:
donga = milvus_connect("Donga")
print(donga)

donga_800 = milvus_connect("Donga_800")
print(donga_800)

Milvus connected
Collection 'Donga' loaded.
<Collection>:
-------------
<name>: Donga
<description>: 
<schema>: {'auto_id': False, 'description': '', 'fields': [{'name': 'id', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 100}, 'is_primary': True, 'auto_id': False}, {'name': 'title', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 2000}}, {'name': 'date', 'description': '', 'type': <DataType.INT64: 5>}, {'name': 'content', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 5000}}, {'name': 'NER', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 3000}}, {'name': 'dense_vector', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 4096}}, {'name': 'sparse_vector', 'description': '', 'type': <DataType.SPARSE_FLOAT_VECTOR: 104>}], 'enable_dynamic_field': False}

Milvus connected
Collection 'Donga_800' loaded.
<Collection>:
-------------
<name>: Donga_800
<desc

## Load Dense & Sparse Embedding Function

In [4]:
# Sparse Embedding : Custom BM25

from model.bm25_model import Custom_BM25, Kiwi_Tokenizer

tokenizer = Kiwi_Tokenizer("model/custom_dict.txt")
custom_bm25 = Custom_BM25(corpus=[], tokenizer=tokenizer)
custom_bm25.load("model/bm25_model.json")
print(custom_bm25)

c:\Users\kidong\anaconda3\envs\milvus_hybrid\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [6]:
# Dense Embedding : Upstage/solar-embedding-1-large
from langchain_upstage import UpstageEmbeddings
from dotenv import load_dotenv
import os

load_dotenv()

# dense_embedding_func = UpstageEmbeddings(
#     model="solar-embedding-1-large-passage",
#     upstage_api_key=os.getenv("UPSTAGE_API_KEY")
# )

dense_embedding_func = UpstageEmbeddings(
    model="solar-embedding-1-large-query",
    upstage_api_key=os.getenv("UPSTAGE_API_KEY")
)

## Retriever

In [38]:
from langchain_milvus.retrievers import MilvusCollectionHybridSearchRetriever
from pymilvus import WeightedRanker

sparse_search_params = {"metric_type": "IP"}
dense_search_params = {"metric_type": "IP", "params": {}}
retriever = MilvusCollectionHybridSearchRetriever(
    collection=donga,
    rerank=WeightedRanker(0.7, 0.3),
    anns_fields=[DENSE_FIELD, SPARSE_FILED],
    field_embeddings=[dense_embedding_func, custom_bm25],
    field_search_params=[dense_search_params, sparse_search_params],
    top_k=5,
    text_field='content',
)
print(retriever)

collection=<Collection>:
-------------
<name>: Donga
<description>: 
<schema>: {'auto_id': False, 'description': '', 'fields': [{'name': 'id', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 100}, 'is_primary': True, 'auto_id': False}, {'name': 'title', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 2000}}, {'name': 'date', 'description': '', 'type': <DataType.INT64: 5>}, {'name': 'content', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 5000}}, {'name': 'NER', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 3000}}, {'name': 'dense_vector', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 4096}}, {'name': 'sparse_vector', 'description': '', 'type': <DataType.SPARSE_FLOAT_VECTOR: 104>}], 'enable_dynamic_field': False}
 rerank=<pymilvus.client.abstract.WeightedRanker object at 0x0000021FAA3A11E0> anns_fields=['dense_vector', 'sparse_vector'] field_em

In [68]:
def hybrid_search(query : str, filter : str = None):
    sparse_search_params = {"metric_type": "IP"}
    dense_search_params = {"metric_type": "IP", "params": {}}

    params = {
        "collection" : donga,
        "rerank" : WeightedRanker(0.7, 0.3),
        "anns_fields" : [DENSE_FIELD, SPARSE_FILED],
        "field_embeddings" : [dense_embedding_func, custom_bm25],
        "field_search_params" : [dense_search_params, sparse_search_params],
        "top_k" : 20,
        "text_field" : 'content',
    }
    if filter:
        params['field_exprs'] = [filter, filter] # 검색 조건 추가(dense와 sparse에 동일하게 적용)
    retriever = MilvusCollectionHybridSearchRetriever(**params)
    
    retrieved_docs = retriever.invoke(query)
    data = []
    unique_id = []
    for retreived_doc in retrieved_docs:
        doc_id = retreived_doc.metadata['id'].split("_")[0] # 기사 고유 ID 추출
        if doc_id not in unique_id: # 해당 기사가 아직 반환 리스트에 없는 경우만
            data.append(retreived_doc)
            unique_id.append(doc_id)
        if len(data) == 5: break # 상위 5개만 반환
    return data

In [71]:
hybrid_search("최근 조국혁신당과 윤석열 대통령 사이의 관계를 알아볼 수 있을 만한 기사를 알려줘")

[Document(metadata={'id': '124474290_1', 'title': '조국당, 광주 득표율 민주연합에 11%P 앞서', 'date': 20240415, 'NER': '조국혁신당 민주연합 조국혁신당 윤석열 이재명 민주당 호남 조국혁신당 국회 민주당 조국혁신당 국민의미래 조국혁신당 더불어민주연합 서울 조국혁신당 더불어민주연합 국민의미래 서초 강남 조국 윤석열 대통령 민주당 이재명 대통령'}, page_content='외한 전 지역에서 조국혁신당이 민주연합을 앞섰다. 이는 조국혁신당이 "3년은 너무 길다"며 윤석열 정권 심판 목소리를 민주연합보다 강하고 선명하게 내왔기 때문이라는 분석이다. 이재명 대표의 민주당에 불만을 가진 호남의 야권 지지층이 조국혁신당을 지지한 것으로도 풀이된다. 이 때문에 향후 22대 국회에서 호남 민심을 놓고 민주당과 조국혁신당 사이에 경쟁을 넘어 묘한 긴장 관계가 형성될 수 있다는 시각도 나온다. 텃밭 외 지역에선 국민의미래가 승리한 곳에서 조국혁신당이 더불어민주연합을 앞서는 현상이 두드러졌다. 서울에선 조국혁신당이 2개 지역에서만 더불어민주연합을 앞섰는데, 국민의미래가 압도적 1등을 한 서초와 강남이었다. 조국 대표는 14일 "원내 제3당의 대표로서 언제, 어떤 형식이건 윤석열 대통령을 만날 수 있길 희망한다"고 밝혔다. 민주당 이재명 대표에 이어 윤 대통령과의 회담 개최를 압박한'),
 Document(metadata={'id': '123826025_1', 'title': '\'이재명-조국 총선연대\' 사실상 공식화… 여 "李 방탄막 세우기 몰두"', 'date': 20240305, 'NER': '조국혁신당 대한민국 민주당 윤석열 윤석열 검찰 조국혁신당 민주당 더불어민주연합 검찰 조기종식 윤석열 대통령 부인 김건희 박정하 수석대변인 자녀 청와대 국회'}, page_content=' 조국혁신당은 현재 대한민국 질곡을 함께 헤쳐 나갈 동지라 생각한다"고 말했다. 그는 "민주당은 윤석열 정권에 실망한 중도파와 합리적 보

In [73]:
hybrid_search("최근 조국혁신당과 윤석열 대통령 사이의 관계를 알아볼 수 있을 만한 기사를 알려줘", filter='date >= 20240601')

[Document(metadata={'id': '125305010_0', 'title': '현충일 추념식서 윤과 악수 나눈 조국…"민심 받들라 말했다"', 'date': 20240606, 'NER': '조국 조국혁신당 윤석열 대통령 대통령 2568년 법요식 조국혁신당 추념식 대통령 대통령 자손 윤석열 정부 만대한민국'}, page_content='조국 조국혁신당 대표가 6일 현충일 추념식에서 윤석열 대통령에게 "민심을 받들라"는 취지로 말한 것으로 전해졌다. 윤 대통령과 조 대표의 만남은 지난달 15일 \'불기 2568년 부처님오신날 봉축 법요식\' 이후 3주 만이다. 조국혁신당에 따르면 조 대표는 이날 국립서울현충원에서 열린 제69회 현충일 추념식에서 윤 대통령과 악수하면서 "민심을 받드십시오"라고 말했다. 이에 윤 대통령은 별말 없이 움찔한 것 같다고 조국혁신당은 주장했다. 앞서 조 대표는 이날 오전 메시지를 내고 "친일(친일), 종일(종일), 숭일(숭일), 부일(부일)하는 모리배·매국노들이 호의호식하고 고위직에 올라 떵떵거리고 사는 일이 없도록 하겠다"며 "애국열사와 유공자들이 제대로 대우 받고 그 유족과 자손들이 떳떳하게 사는 나라로 예인하겠다"고 밝혔다. 그러면서 윤석열 정부를 향해 "불과 2년 만에 대한민국은 40년, 50년, 6'),
 Document(metadata={'id': '125225554_0', 'title': '조국당 "축하난 거부가 옹졸? \'거부왕\' 윤이 쫄보"', 'date': 20240601, 'NER': '조국혁신당 의원 윤석열 대통령 대통령 김보협 수석대변인 앞 수석대변인 용산 대통령 국민의힘 조국혁신당 대통령 정무수석 가족 거부권 윤 대통령 수석대변인 대통령 계란말이 김치찌개 대파'}, page_content='조국혁신당은 자당 의원들이 윤석열 대통령의 당선 축하 난(난)을 거부한 것에 대해 국민의힘이 \'옹졸한 정치\'라고 비판하자 "\'거부왕\' 윤 대통령이 옹졸한 정치"라고 반박했다. 김보협 수석대변인은 1일 논평을

## Comparing With previous version(dense embedding only w/ 800 characters)

In [75]:
from openai import OpenAI
import numpy as np

INDEX_PARAMS = {
    'metric_type': 'COSINE',     # "L2", "IP", "COSINE" 
    'index_type': "IVF_FLAT",    # "IVF_FLAT","IVF_SQ8", "IVF_PQ", "HNSW", "ANNOY"
    'params': {"nlist": 128},
}

client = OpenAI(
    api_key = os.getenv("UPSTAGE_API_KEY"),
    base_url = "https://api.upstage.ai/v1/solar"
)

def vector_search(collection, query, index_params, option=None):
    # Query embedding 생성
    query_embedding = client.embeddings.create(
        model="solar-embedding-1-large-query",
        input=query
    ).data[0].embedding
    query_embedding = np.asarray(query_embedding)
    
    # 공통 search 파라미터 설정
    search_params = {
        "data": [query_embedding],
        "anns_field": 'embedding',
        "param": index_params,
        "limit": 5,
        "output_fields": ['title', 'date', 'content', 'link']
    }

    # 옵션이 있는 경우 expr 추가
    if option:
        search_params["expr"] = option

    # 검색 실행
    result = collection.search(**search_params)

    # 결과 처리
    result_list = []
    for hits in result:
        for hit in hits:
            result_list.append(hit.to_dict())
    
    return result_list

In [76]:
vector_search(donga_800, "최근 조국혁신당과 윤석열 대통령 사이의 관계를 알아볼 수 있을 만한 기사를 알려줘", INDEX_PARAMS)

[{'id': 451472232637362863,
  'distance': 0.49557554721832275,
  'entity': {'title': '조국당 “축하난 거부가 옹졸? ‘거부왕’ 尹이 쫄보”',
   'date': 20240601,
   'content': '조국혁신당은 자당 의원들이 윤석열 대통령의 당선 축하 난을 거부한 것에 대해 국민의힘이 옹졸한 정치라고 비판하자 거부왕 윤 대통령이 옹졸한 정치라고 반박했다. 김보협 수석대변인은 1일 논평을 통해 의원실 앞에 몰래 난 화분을 놓고 가는 행위를 협치로 보는 국민은 없을 것이라고 비판했다. 김 수석대변인은 용산 대통령실 혹은 여당인 국민의힘 그 누구라도 조국혁신당에 만남이나 대화를 제안한 적이 있느냐며 창당한 지 석 달이 다 돼가는데도 대통령실 정무수석은 코빼기도 보이지 않는다고 지적했다. 이어 자신과 가족을 위해 거부권을 남발하는 거부왕 윤 대통령만큼 옹졸한 정치를 잘 보여주는 이는 없을 것이라고 주장했다. 김 수석대변인은 또 출입기자단 초청 대통령과의 만찬 메뉴였던 계란말이와 김치찌개에 대파가 빠졌다던데 그런 게 옹졸 쫄보의 상징이라고 비꼬았다. 조국혁신당 조국 대표는 지난달 31일 페이스북에 윤 대통령의 축하 난 사진과 함께 역대 유례없이 사익을 위하여 거부권을 오남용하는 대통령의 축하 난은 정중히 사양한다고 썼다. 윤 대통령은 지난달 30일 22대 국회 당선인 300명 전원에게 국회의원 당선을 축하합니다. 대통령 윤석열이라고 적힌 리본이 달린 난을 보냈다. 문재인 정부 국립외교원장 출신인 김준형 의원은 불통령불통대통령의 합성어의 난을 버린다고 인증했다. 김 의원은 버립니다라고 적힌 메모지를 부착한 축하 난 사진도 함께 올렸다. 같은 당 차규근 의원은 리본 가운데 대통령 윤석열이라고 적힌 부분을 가위로 잘라낸 사진을 자신의 페이스북에 올렸다. 당론 1호 법안인 한동훈 특검법을 대표 발의한 박은정 의원은 잘 ',
   'link': 'https://www.donga.com/news/Poli

In [77]:
vector_search(donga_800, "임성근 전 해병대 1사단장과 관련된 이슈를 다룬 기사를 알고 싶어.", INDEX_PARAMS)

[{'id': 451472232637377869,
  'distance': 0.5360703468322754,
  'entity': {'content': '도이치모터스 주가 조작 사건으로 재판을 받고 있는 이모 씨블랙펄인베스트먼트 전 대표가 임성근 전 해병대1사단장의 구명 로비 의혹에 대해 녹취록을 제보한 변호사가 의도를 가지고 내용을 퍼뜨린 것이라고 일축했다. 이 씨는 변호사가 공개한 녹취록에 담긴 도 윤석열 대통령이 아닌 김계환 해병대 사령관이라는 입장이다. 이 씨는 10일 동아일보 기자와 만나 녹취록의 언급이 마치 제가 한 이야기처럼 보도가 됐는데 같은 카카오톡 단체 채팅방에 있던 멤버 씨와 통화한 것을 변호사에게 전달해 말한 것 뿐이라고 말했다. 채 상병 순직 수사 외압 의혹을 수사 중인 고위공직자범죄수사처공수처는 이 씨가 임성근 전 해병대 1사단장을 구명했다고 주변에 자랑했다는 취지의 진술과 전화 녹음 파일을 확보한 것으로 알려졌다. 동아일보가 확보한 변호사와 이 씨의 통화 녹음파일에 따르면 이 씨는 임 전 사단장이 사표를 낸다고 그래 가지고 씨가 전화 왔더라고. 그래 가지고 내가 절대 사표 내지 마라 내가 한테 얘기를 하겠다라고 말했다. 변호사가 올해 3월 통화에서도 임 전 사단장이 채 상병 순직 사건에 책임이 있는 것 같다고 하자 이 씨는 그러니까 쓸데없이 내가 거기 개입이 돼 가지고 사표 낸다고 그럴 때 내라 그럴걸이라고 말한 것으로 확인됐다. 이에 대해 이 씨는 이날 동아일보에 채상병 사건이 일어나고 해병대 후배인 씨가 임 전 사단장이 힘들어 한다. 극단 선택할 것 같다. 나쁜 생각 말라고 이렇게 보냈는데 한 번 봐주시라며 메시지를 보내왔다며 녹취록에 언급된 상황에 대해 설명했다. 씨는 이 씨와 변호사가 함께 있는 카카오톡 단체 대화방의 멤버로 ',
   'link': 'https://www.donga.com/news/Politics/article/all/20240710/125864805/1',
   'title': '[단독]‘도이치’ 이모 씨

In [78]:
hybrid_search("임성근 전 해병대 1사단장과 관련된 이슈를 다룬 기사를 알고 싶어.", filter='date >= 20240601')

[Document(metadata={'id': '125864805_0', 'title': '[단독]\'도이치\' 이모 씨 "\'VIP\'는 대통령 아닌 김계환, 김여사 번호도 몰라"…구명 로비 의혹 부인', 'date': 20240710, 'NER': '도이치모터스 이모 블랙펄인베스트먼트 임성근 해병대1사단장 변호사 변호사 윤석열 대통령 김계환 해병대 사령관 동아일보 기자 카카오톡 변호사 상병 고위공직자범죄수사처 공수처 임성근 해병대 1사단장 동아일보 변호사'}, page_content='도이치모터스 주가 조작 사건으로 재판을 받고 있는 이모 씨(블랙펄인베스트먼트 전 대표)가 임성근 전 해병대1사단장의 \'구명 로비 의혹\'에 대해 "녹취록을 제보한 A 변호사가 의도를 가지고 내용을 퍼뜨린 것"이라고 일축했다. 이 씨는 A 변호사가 공개한 녹취록에 담긴 \'VIP\'도 윤석열 대통령이 아닌 김계환 해병대 사령관이라는 입장이다. 이 씨는 10일 동아일보 기자와 만나 "(녹취록의 \'VIP\' 언급이) 마치 제가 한 이야기처럼 보도가 됐는데 같은 (카카오톡 단체 채팅)방에 있던 멤버 B 씨와 통화한 것을 A 변호사에게 전달해 말한 것 뿐"이라고 말했다. 채 상병 순직 수사 외압 의혹을 수사 중인 고위공직자범죄수사처(공수처)는 이 씨가 "임성근 전 해병대 1사단장을 구명했다"고 주변에 자랑했다는 취지의 진술과 전화 녹음 파일을 확보한 것으로 알려졌다. 동아일보가 확보한 A 변호사와 이 씨의 통화'),
 Document(metadata={'id': '126061649_0', 'title': '[단독]임성근, 청문회 증언 이튿날 국회에 \'증언 정정\' 진술서 제출… "확인 결과 송씨 등 6명 추가 초청 지시"', 'date': 20240723, 'NER': '국회 공수처 임성근 해병대 1사단장 대통령경호처 국회 국회 동아일보 사단장 해병대 국회 해병대 해병대 상병 사단장 사단장 골프 카카오톡 국회 법제사법위원회 윤석열 대통령 더불어민주당 장경태 의원 사단장 해병대 1